In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from nltk.stem import PorterStemmer
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import jieba
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile
import sklearn
import scipy
from gensim.test.utils import datapath
from gensim.corpora import Dictionary
from gensim.corpora import MalletCorpus
from gensim.test.utils import get_tmpfile, common_texts
from gensim.models.wrappers import LdaMallet
from gensim.test.utils import common_corpus, common_dictionary
import time
from collections import defaultdict
from random import random, randint
from glob import glob
from math import log
import argparse
import random

from nltk.corpus import stopwords
from nltk.probability import FreqDist

from nltk.tokenize import TreebankWordTokenizer
kTOKENIZER = TreebankWordTokenizer()
kDOC_NORMALIZER = True

In [42]:
search_path = "%s/*.txt" % './w2v/lit/en'

files = glob(search_path)
print(files)

documents = []

for i in files:
    contents = open(i).read()
    documents.append(contents)


texts = [
    [word for word in jieba.cut(document)] for document in documents
]


print(len(texts))

model_en = Word2Vec(texts, size=100, window=5, min_count=1, workers=4)
model_en.train(texts,total_examples=model_en.corpus_count, epochs=100)


search_path = "%s/*.txt" % './w2v/lit/cn'

files = glob(search_path)
print(files)


documents = []

for i in files:
    contents = open(i).read()
    documents.append(contents)


texts = [
    [word for word in jieba.cut(document)] for document in documents
]


print(len(texts))

model_cn = Word2Vec(texts, size=100, window=5, min_count=1, workers=4)
model_cn.train(texts,total_examples=model_cn.corpus_count, epochs=100)


['./w2v/lit/en/2e.txt']
1
['./w2v/lit/cn/2c.txt']
1


(1000000, 15105900)

In [3]:
def get_similar_vector(model_i,word):
    try:
        word = model_i.wv.most_similar(positive=[model_i.wv[word]], topn=2)[1][0]
        most_similar_wv = model_i.wv[word]
    except:
        most_similar_wv = model_i.wv[random.choice(model_i.wv.index2entity)]
    return most_similar_wv


def get_word_for_m2_if_m1_i(model_1,model_2,word):
    try:
        word = model_2.wv.most_similar(positive=[model_1.wv[word]], topn=1)[0][0]
        most_similar = word
    except:
#         wv = model_2.wv[random.choice(model_2.wv.index2entity)]
        most_similar = random.choice(list(model_2.wv.vocab))
#         print("r",end="")
#         most_similar = model_2.most_similar(positive=wv, topn=1)[0][0]
        
    return most_similar


# def get_word_for_m2_if_m1_i(model_1,model_2,word):
#     most_similar = random.choice(list(model_2.wv.vocab))

#     return most_similar

In [4]:
# list(model_cn.wv.vocab)
random.choice(list(model_cn.wv.vocab))

'扔掉'

In [5]:

# try:
#     word = model.wv.most_similar(positive=[model.wv['pride']], topn=2)[1][0]
#     most_similar_wv = model.wv[word]
# except:
#     most_similar_wv = model.wv[random.choice(model.wv.index2entity)]
    
# print(get_similar_vector(model_en,'pridedd'))


get_word_for_m2_if_m1_i(model_cn,model_en,'年末')

'今冬'

In [6]:
tw_pd_en = pd.read_csv("./base_line_classifier/tra.csv",sep=',',encoding='latin-1',names=['class','ram','time','q','user_id','text'])

train_S = tw_pd_en['class']
train_ST = tw_pd_en['text']

def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text


def tokenizer(text):
    words = text.split()
#     words = [get_word_for_m2_if_m1_i(model_cn,model_en,word) for word in words]
    return words

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]



# tokenizer('A good day')

In [7]:
stop = stopwords.words('english')

# split the dataset in train and test
X = train_ST
y = train_S
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__tokenizer': [tokenizer],
               'vect__preprocessor': [ preprocessor],
               'clf__penalty': ['l2'],
               'clf__C': [1.0]},
              ]

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=3)

gs_lr_tfidf.fit(X_train, y_train)

print('Best parameter set: ' + str(gs_lr_tfidf.best_params_))
print('Best accuracy: %.3f' % gs_lr_tfidf.best_score_)

clf = gs_lr_tfidf.best_estimator_
print('Accuracy in test: %.3f' % clf.score(X_test, y_test))


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:  4.0min finished
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Best parameter set: {'clf__C': 1.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__preprocessor': <function preprocessor at 0x7f0ef3a62290>, 'vect__tokenizer': <function tokenizer at 0x7f0ef3a62440>}
Best accuracy: 0.802
Accuracy in test: 0.803


In [16]:
get_word_for_m2_if_m1_i(model_cn,model_en,'年末')

'twenty'

In [17]:
tw_pd_cn = pd.read_csv("./base_line_classifier/cna.csv",sep=',',encoding='utf-8',names=['weibo_id','time','id_name','text','ul','ul2','class'])

train_S_cn = tw_pd_cn['class'].iloc[1:]
train_ST_cn = tw_pd_cn['text'].iloc[1:]

# train_ST_cn

In [52]:
len( tw_pd_cn)

100001

In [50]:
def tokenizer_cn(text):
    words = list(jieba.cut(text))
    words = [get_word_for_m2_if_m1_i(model_cn,model_en,word) for word in words]

    text=  ' '.join(words)

    return text



# x_begin = 14000
x_begin = 0
length = 300
X_cn_test = train_ST_cn.iloc[x_begin+0:x_begin+length].apply(tokenizer_cn)
y_cn_test = train_S_cn.iloc[x_begin+0:x_begin+length]


In [51]:
pred = gs_lr_tfidf.predict(X_cn_test)
# pred= [int(i) for i in pred/4]

# print(pred)
y_cn_test = list(y_cn_test)
y_cn_test = [int(i) for i in y_cn_test]

# print(y_cn_test)

n,m=0,0

for i,v in enumerate(pred):
    if pred[i]==4 and y_cn_test[i]!=1:
        m+=1
    else:
        n+=1
print(n/(n+m))

right,wrong=0,0

for i,v in enumerate(y_cn_test):
    if y_cn_test[i]==0:
        continue
    if y_cn_test[i]==1 and pred[i]==4:
        right+=1
    elif y_cn_test[i]==-1 and pred[i]==0:
        right+=1
    else:
        wrong+=1
        
print(right/(right+wrong))

0.31666666666666665
0.3150684931506849


In [12]:
pred = gs_lr_tfidf.predict(X_test)

from collections import Counter 

c = Counter(pred == y_test)
c[True]/(c[True]+c[False])

0.8026979166666667